<a href="https://colab.research.google.com/github/germanilia/ai_and_stuff/blob/main/webinar_open_ai/demo/agent_tools_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#@title Set API Keys
import os
os.environ["AZURE_OPENAI_API_KEY"] = ""
os.environ["SERPAPI_API_KEY"] = ""

In [8]:
#@title Install dependencies
!pip -q install langchain huggingface_hub openai google-search-results tiktoken wikipedia

In [9]:
#@title Set env variables
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["AZURE_MODEL_NAME"] = "text-davinci-003"
os.environ["AZURE_DEPLOYMENT_NAME"] = "text-davinci-003"
os.environ["OPENAI_API_VERSION"] = "2022-12-01"
os.environ["OPENAI_API_BASE"] = "https://ilia.openai.azure.com"
os.environ["OPENAI_API_KEY"] = os.environ.get("AZURE_OPENAI_API_KEY")

In [10]:
#@title Initilize LLM
import os
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(
    deployment_name=os.environ.get("AZURE_DEPLOYMENT_NAME"),
    model_name=os.environ.get("AZURE_MODEL_NAME"),
    temperature=0,
    streaming=False,
)


In [11]:
res = llm("tell me about yourself")
print(res)



I'm a recent college graduate with a degree in Business Administration. I'm passionate about learning new things and exploring new opportunities. I'm an organized and detail-oriented person who loves to take on challenges and solve problems. I'm also a creative thinker who enjoys coming up with innovative solutions. In my free time, I enjoy reading, playing sports, and spending time with friends and family.


In [27]:
#@title Load Tools
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [28]:
# What is a tool like this
tools[1].name, tools[1].description

('Calculator', 'Useful for when you need to answer questions about math.')

In [29]:
#@title Initializing agent
agent = initialize_agent(tools, 
                         llm, 
                         agent="zero-shot-react-description", 
                         verbose=True)

In [ ]:
agent.agent.llm_chain.prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\nSearch: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator: Useful for when you need to answer questions about math.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search, Calculator]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

Answer the following questions as best you can. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}

In [ ]:
agent.run("Hi How are you today?")



> Entering new AgentExecutor chain...
 This is not a question that can be answered with a search or a calculator.
Action: None
Action Input: None
Observation: None is not a valid tool, try another one.
Thought: This is a question that requires a personal response.
Final Answer: I'm doing well, thank you for asking!

> Finished chain.


"I'm doing well, thank you for asking!"

In [ ]:
agent.run("Who is the United States President? What is his current age raised divided by 2?")




> Entering new AgentExecutor chain...
 I need to find out who the President is and then do some math.
Action: Search
Action Input: "United States President"
Observation: Joe Biden
Thought: I now need to find out Joe Biden's age.
Action: Search
Action Input: "Joe Biden age"
Observation: 80 years
Thought: I now need to divide Joe Biden's age by 2.
Action: Calculator
Action Input: 80/2
Observation: Answer: 40.0
Thought: I now know the final answer.
Final Answer: Joe Biden is the United States President and his current age raised divided by 2 is 40.0.

> Finished chain.


'Joe Biden is the United States President and his current age raised divided by 2 is 40.0.'

In [1]:
agent.run("Who is the current CEO of sela cloud from israel?")

NameError: ignored

In [18]:
#@title New Agent
tools = load_tools(["serpapi", "llm-math","wikipedia","terminal"], llm=llm)
agent = initialize_agent(tools, 
                         llm, 
                         agent="zero-shot-react-description", 
                         verbose=True)


In [ ]:
agent.run("Who is the head of DeepMind")



> Entering new AgentExecutor chain...
 I need to find out who is the head of DeepMind
Action: Search
Action Input: "head of DeepMind"
Observation: CEO Demis Hassabis
Thought: I now know the final answer
Final Answer: Demis Hassabis is the head of DeepMind.

> Finished chain.


'Demis Hassabis is the head of DeepMind.'

In [23]:
agent.run("What is DeepMind")



> Entering new AgentExecutor chain...
 DeepMind is a company, so I should look it up on Wikipedia.
Action: Wikipedia
Action Input: DeepMind

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Google DeepMind
Summary: Google DeepMind, formerly DeepMind Technologies, is an artificial intelligence research laboratory which serves as a subsidiary of Google. It was originally founded in 2010 in the United Kingdom before being acquired by Google in 2014, becoming a wholly owned subsidiary of Google parent company Alphabet Inc. after Google's corporate restructuring in 2015. The company is based in London, with research centres in Canada, France, and the United States.
Google DeepMind has created a neural network that learns how to play video games in a fashion similar to that of humans, as well as a Neural Turing machine, or a neural network that may be able to access an external memory like a conventional Turing machine, resulting in a computer that mimics the short-term memory of the human brain.DeepMind made headlines in 2016 after its AlphaGo program beat a human professional Go player Lee Sedol, a world champion, in a five-game match, which was the subjec

'AlphaGo is a computer program developed by the London-based DeepMind Technologies, an acquired subsidiary of Google (now Alphabet Inc.). It uses a Monte Carlo tree search algorithm to find its moves based on knowledge previously acquired by machine learning, specifically by an artificial neural network (a deep learning method) by extensive training, both from human and computer play. In October 2015, it became the first computer Go program to beat a human professional Go player without handicap on a full-sized 19×19 board. In March 2016, it beat Lee Sedol in a five-game match, the first time a computer Go program has beaten a 9-dan professional without handicap.'

In [ ]:
agent.run("Does my current directory have a file about California?")



> Entering new AgentExecutor chain...
 I need to find out what files are in my current directory.
Action: terminal
Action Input: ls
Observation: sample_data

Thought:

/usr/local/lib/python3.10/dist-packages/langchain/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


 I need to find out if there is a file about California in this directory.
Action: terminal
Action Input: grep California sample_data
Observation: grep: sample_data: Is a directory

Thought: I need to look inside the sample_data directory to see if there is a file about California.
Action: terminal
Action Input: ls sample_data
Observation: anscombe.json
california_housing_test.csv
california_housing_train.csv
mnist_test.csv
mnist_train_small.csv
README.md

Thought: I now know the final answer.
Final Answer: Yes, there is a file about California in my current directory.

> Finished chain.


'Yes, there is a file about California in my current directory.'